<a href="https://colab.research.google.com/github/yunseo-qleap/aeqis/blob/inhyuk/noise_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 4.8 MB/s 
     |████████████████████████████████| 18.0 MB 16.6 MB/s 
     |████████████████████████████████| 240 kB 38.8 MB/s 
     |████████████████████████████████| 198 kB 49.6 MB/s 
     |████████████████████████████████| 112 kB 18.2 MB/s 
     |████████████████████████████████| 54 kB 1.1 MB/s 
     |████████████████████████████████| 1.6 MB 411 kB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 943 kB 46.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
     |████████████████████████████████| 4.0 MB 29.8 MB/s 
     |████████████████████████████████| 112 kB 55.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.36.2-py3-none-any.whl size=11933 sha256=4b2c9917cc

In [2]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.3 MB/s 
     |████████████████████████████████| 9.3 MB 16.9 MB/s 
     |████████████████████████████████| 108 kB 28.6 MB/s 


In [3]:
import pennylane as qml
from pennylane import numpy as np

In [4]:
dev = qml.device('default.mixed', wires=2)


In [18]:
ev = np.tensor(0.7781, requires_grad=False)  # observed expectation value

def sigmoid(x):
    return 1/(1+np.exp(-x))

@qml.qnode(dev)
def damping_circuit(x):
    qml.Hadamard(wires=0)
    for i in range(8):
       qml.CNOT(wires=[i, i+1])
       qml.AmplitudeDamping(sigmoid(x), wires=i)  # p = sigmoid(x) 
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [16]:
def cost(x, target):
    return (damping_circuit(x) - target)**2

In [17]:
opt = qml.GradientDescentOptimizer(stepsize=10)
steps = 35
x = np.tensor(0.0, requires_grad=True)

for i in range(steps):
    (x, ev), cost_val = opt.step_and_cost(cost, x, ev)
    if i % 5 == 0 or i == steps - 1:
        print(f"Step: {i}    Cost: {cost_val}")

print(f"QNode output after optimization = {damping_circuit(x):.4f}")
print(f"Experimental expectation value = {ev}")
print(f"Optimized noise parameter p = {sigmoid(x.take(0)):.4f}")

/usr/local/lib/python3.7/dist-packages/autograd/numpy/numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


Step: 0    Cost: 0.07733961000000007
Step: 5    Cost: 0.07733961000000007
Step: 10    Cost: 0.07733961000000007
Step: 15    Cost: 0.07733961000000007
Step: 20    Cost: 0.07733961000000007
Step: 25    Cost: 0.07733961000000007
Step: 30    Cost: 0.07733961000000007
Step: 34    Cost: 0.07733961000000007
QNode output after optimization = 0.5000
Experimental expectation value = 0.7781
Optimized noise parameter p = 0.5000


In [94]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[0.50097656 0.         0.         ... 0.         0.         0.02209709]
 [0.         0.00097656 0.         ... 0.         0.         0.        ]
 [0.         0.         0.00097656 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.00097656 0.         0.        ]
 [0.         0.         0.         ... 0.         0.00097656 0.        ]
 [0.02209709 0.         0.         ... 0.         0.         0.00097656]]


QNode output = 1.0000


In [102]:
import matplotlib.pyplot as plt
fig = qml.draw_mpl(dev)
plt.show()

In [5]:
@qml.qnode(dev)
def bitflip_circuit(p):
    qml.Hadamard(wires=0)
    for i in range(8):
      qml.CNOT(wires=[i, i+1])
      qml.BitFlip(p, wires=i)

    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


ps = [0.001, 0.01, 0.1, 0.2]
for p in ps:
    print(f"QNode output for bit flip probability {p} is {bitflip_circuit(p):.4f}")

WireError: ignored

In [106]:
print(f"Output state for bit flip probability {p} is \n{np.real(dev.state)}")

Output state for bit flip probability 0.2 is 
[[8.388608e-02 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  8.388608e-02]
 [0.000000e+00 1.280000e-06 0.000000e+00 ... 0.000000e+00 1.280000e-06
  0.000000e+00]
 [0.000000e+00 0.000000e+00 2.097152e-02 ... 2.097152e-02 0.000000e+00
  0.000000e+00]
 ...
 [0.000000e+00 0.000000e+00 2.097152e-02 ... 2.097152e-02 0.000000e+00
  0.000000e+00]
 [0.000000e+00 1.280000e-06 0.000000e+00 ... 0.000000e+00 1.280000e-06
  0.000000e+00]
 [8.388608e-02 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  8.388608e-02]]


In [6]:
ev = np.tensor(0.7781, requires_grad=False)  # observed expectation value

def sigmoid(x):
    return 1/(1+np.exp(-x))

@qml.qnode(dev)
def damping_circuit(x):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.AmplitudeDamping(sigmoid(x), wires=0)  # p = sigmoid(x)
    qml.AmplitudeDamping(sigmoid(x), wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [7]:

def cost(x, target):
    return (damping_circuit(x) - target)**2

In [111]:
opt = qml.GradientDescentOptimizer(stepsize=10)
steps = 35
x = np.tensor(0.0, requires_grad=True)

for i in range(steps):
    (x, ev), cost_val = opt.step_and_cost(cost, x, ev)
    if i % 5 == 0 or i == steps - 1:
        print(f"Step: {i}    Cost: {cost_val}")

print(f"QNode output after optimization = {damping_circuit(x):.4f}")
print(f"Experimental expectation value = {ev}")
print(f"Optimized noise parameter p = {sigmoid(x.take(0)):.4f}")

/usr/local/lib/python3.7/dist-packages/autograd/numpy/numpy_wrapper.py:156: ComplexWarning: Casting complex values to real discards the imaginary part
  return A.astype(dtype, order, casting, subok, copy)


Step: 0    Cost: 0.07733961000000007
Step: 5    Cost: 0.07733961000000007
Step: 10    Cost: 0.07733961000000007
Step: 15    Cost: 0.07733961000000007
Step: 20    Cost: 0.07733961000000007
Step: 25    Cost: 0.07733961000000007
Step: 30    Cost: 0.07733961000000007
Step: 34    Cost: 0.07733961000000007
QNode output after optimization = 0.5000
Experimental expectation value = 0.7781
Optimized noise parameter p = 0.5000


In [112]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[0.625 0.    0.    0.25 ]
 [0.    0.125 0.    0.   ]
 [0.    0.    0.125 0.   ]
 [0.25  0.    0.    0.125]]


In [32]:
import pennylane as qml
from pennylane import numpy as np

dev = qml.device('default.mixed', wires=9)


@qml.qnode(dev)
def bitflip_circuit(p):
    qml.Hadamard(wires=0)
    for i in range(8):

      qml.CNOT(wires=[i, i+1])
      qml.BitFlip(p, wires=i)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


p = 0.01
print(f"QNode output for bit flip probability {p} is {bitflip_circuit(p):.4f}")

QNode output for bit flip probability 0.01 is 0.9604


In [33]:
print(f"Output state is = \n{np.real(dev.state)}")

Output state is = 
[[4.61372347e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 4.61372347e-01]
 [0.00000000e+00 5.00000000e-17 0.00000000e+00 ... 0.00000000e+00
  5.00000000e-17 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.66032674e-03 ... 4.66032674e-03
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 4.66032674e-03 ... 4.66032674e-03
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.00000000e-17 0.00000000e+00 ... 0.00000000e+00
  5.00000000e-17 0.00000000e+00]
 [4.61372347e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 4.61372347e-01]]
